# Formulación del Problema de Asignación de Lotes de Caña a Ingenios

Considere un conjunto de 10 lotes, que pueden ser cosechados y molidos en uno de dos Ingenios: Pánuco o El Mante.

***A dónde deben transportarse para poder maximizar la cantidad de azúcar producida?***

## Ejemplo de Datos

Se realizó un muestreo a distintos lotes de productores de caña en las cercanías de ambos ingenios, para obtener una estimación del rendimiento de azúcar. Así mismo, cual es el costo estimado de entregar la caña en un ingenio versus el otro. Los datos se resumen a continuación:


In [1]:
# librerías
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('lotes.csv')




## Descripción del Problema

Determine una asignación que asegure que cada lote sea molido en alguno de los Ingenios El Mante o Pánuco, y que cada lote se asigne a lo más a un Ingenio para *maximizar* la cantidad de azúcar total para el grupo.

## Variables de Decisión

Se define la variable de decisión $x_{l,\; j} = 1$ que representa que el lote $l$ es asignado al Ingenio $j$, y 0 en otro caso, para todos los lotes $l=1,2,...,10$ y los Ingenios $𝑗=1,2$, en donde $j=1$ para Ingenio El Mante y $j=2$ para Ingenio Pánuco.

## Restricciones

### Restricciones sobre Lotes


Para cada lote $l=1,2,...,10$, exactamente un Ingenio $𝑗=1,2$ puede ser asignado.


Restricción (Lote=1): $\sum_{j=1}^{2} x_{1,\; j} = 1, \forall j \in \{1,2\}$

Restricción (Lote=2): $\sum_{j=1}^{2} x_{2,\; j} = 1, \forall j \in \{1,2\}$

$\vdots$

Restricción (Lote=10): $\sum_{j=1}^{2} x_{10,\; j} = 1, \forall j \in \{1,2\}$

Y finalmente,
$x_{l,j} \in \{0,1\}, \forall l \in \{1,2,...,10\} , \forall j \in \{1,2\}$

## Parámetros
- Sea $Marginal_j$ el beneficio marginal (USD) de la tonelada de azúcar en el ingenio $j$
- Sea $Recuperacion_j$ la recuperación global industrial (%) del ingenio $j$
- Sea $Precio_{l,j}$ el precio de CAT por tonelada de caña del lote $l$ hacia el ingenio $j$



## Función Objetivo

Se formula una función objetivo para *maximizar* el beneficio económico total de las asignaciones de lotes mientras se satisfacen las restricciones de lotes.

$\begin{align}
\max\limits_{\forall j \in \{1,2\}, \forall l \in \{1,2,...,10\}} \sum_{l=1}^{10} \sum_{j=1}^{2} (\dfrac{Caña_l * Core_j}{1000} * Recuperacion_j * Marginal_j - Precio_{l,j} * Caña_l) x_{l,j}
\end{align}$


In [3]:
# Gurobi
from gurobipy import *

## Datos
En una lista 'J' se colectan los nombres de los Ingenios: Mante y Panuco.

En una lista 'L' se colectan los identificadores de los lotes disponibles: {1,2,...,10}.


**Notación Matemática**

$l \in L$ significa que un lote con índice $l$ está en el conjunto (lista) 'L'.

$j \in J$ significa que un Ingenio con índice $j$ está en el conjunto (lista) 'J'.

In [4]:
# conjuntos de lotes e Ingenios
J = ['Mante', 'Panuco']
L = ['Lote 1','Lote 2','Lote 3','Lote 4','Lote 5','Lote 6','Lote 7','Lote 8','Lote 9','Lote 10']

In [5]:
# archivo de datos
df

,Lote,Caña (t),Core (kg/t),Precio Mante ($/t),Precio Panuco ($/t)
0,1,300,105,20,18
1,2,250,130,22,30
2,3,500,120,21,13
3,4,120,115,30,22
4,5,130,98,35,10
5,6,180,135,18,40
6,7,340,100,15,5
7,8,230,110,40,32
8,9,80,115,34,22
9,10,100,138,22,18


In [6]:
# beneficio marginal por tonelada de azúcar
marginal = 390
recuperacion_1 = 0.77
recuperacion_2 = 0.82

# revenue
df_mante = df.iloc[:,:4]
df_mante['revenue'] = df['Caña (t)']*df['Core (kg/t)']*marginal*recuperacion_1/1000 - df_mante['Precio Mante ($/t)']*df_mante['Caña (t)']
df_panuco = df.drop('Precio Mante ($/t)',axis=1)
df_panuco['revenue'] = df['Caña (t)']*df['Core (kg/t)']*marginal*recuperacion_2/1000 - df_panuco['Precio Panuco ($/t)']*df_panuco['Caña (t)']

### Diccionarios de Posibles Asignaciones

In [7]:
# tabla de puntuaciones y costos por recurso y posición
combinations, caña, core, precio, revenue = multidict({
    ('Lote 1', 'Mante'): df_mante.iloc[0,1:].values,
    ('Lote 1', 'Panuco'): df_panuco.iloc[0,1:].values,
    ('Lote 2', 'Mante'): df_mante.iloc[1,1:].values,
    ('Lote 2', 'Panuco'): df_panuco.iloc[1,1:].values,
    ('Lote 3', 'Mante'): df_mante.iloc[2,1:].values,
    ('Lote 3', 'Panuco'): df_panuco.iloc[2,1:].values,
    ('Lote 4', 'Mante'): df_mante.iloc[3,1:].values,
    ('Lote 4', 'Panuco'): df_panuco.iloc[3,1:].values,
    ('Lote 5', 'Mante'): df_mante.iloc[4,1:].values,
    ('Lote 5', 'Panuco'): df_panuco.iloc[4,1:].values,
    ('Lote 6', 'Mante'): df_mante.iloc[5,1:].values,
    ('Lote 6', 'Panuco'): df_panuco.iloc[5,1:].values,
    ('Lote 7', 'Mante'): df_mante.iloc[6,1:].values,
    ('Lote 7', 'Panuco'): df_panuco.iloc[6,1:].values,
    ('Lote 8', 'Mante'): df_mante.iloc[7,1:].values,
    ('Lote 8', 'Panuco'): df_panuco.iloc[7,1:].values,
    ('Lote 9', 'Mante'): df_mante.iloc[8,1:].values,
    ('Lote 9', 'Panuco'): df_panuco.iloc[8,1:].values,
    ('Lote 10', 'Mante'): df_mante.iloc[9,1:].values,
    ('Lote 10', 'Panuco'): df_panuco.iloc[9,1:].values
})


La siguiente función genera un objeto modelo vacío "m" y toma como argumento el nombre 'LAP' (Lot Assignment Problem)

In [8]:
# Se declara e inicializa el modelo
m = Model('LAP')

Set parameter Username


## Variables de Decisión

La variable de decisión $x_{l,\; j} = 1$ representa que el lote $l$ es asignado al Ingenio $j$, y 0 en otro caso, para $l=1,2,...,10$ Y $𝑗=1,2$.

El método “addVars()” define las variables de decisión del objeto modelo "m".

**Notación Matemática**

Sea $x_{l,\; j} = 1$ si el lote $l \in L$  se asigna al Ingenio $j \in J$, y cero en otro caso.

Sea $g_{l} = 1$ si el lote $l \in L$ no puede ser asignado, y cero en otro caso. Esta variable es una variable "gap" que indica que un lote no puede ser asignado a algún Ingenio.


In [9]:
# Crear las variables de decisión para el modelo LAP
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

# Crear las variables "gap" del modelo LAP
g = m.addVars(L, name="gap")

## Resticciones de posición

Para cada lote $l=1,2,...,10$, exactamente un Ingenio de $j=1,2$ debe ser asignado.

**Notación matemática**

Restricción (Lote=$l$): 
$$\sum_{j=1}^{2} x_{l,\; j} + g_{l} = 1, \forall j \in \{1,2\} , \forall l \in \{1,2,...,10\}$$

El método “addConstrs()” define las restricciones del objeto modelo "m".

In [10]:
# crear resticciones de posición
lotes = m.addConstrs((x.sum(l,'*') + g[l]  == 1 for l in L), 'lote')

## Función Objetivo

La función objetivo se plantea para maximizar el revenue derivado de todas las asignaciones de lotes a Ingenios.

**Notación matemática**

$\begin{align}
\max\limits_{\forall j \in \{1,2\}, \forall l \in \{1,2,...,10\}} \sum_{l=1}^{10} \sum_{j=1}^{2} (\dfrac{Caña_l * Core_j}{1000} * Recuperacion_j * Marginal_j - Precio_{l,j} * Caña_l) x_{l,j}
\end{align}$

El método “setObjective()” define la función objetivo del objeto modelo "m".

In [11]:
# Penalización por no asignar algún lote
BIGM =101

# El objetivo es maximizar el revenue total de las asignaciones de lotes a Ingenios
m.setObjective(x.prod(revenue) - BIGM*g.sum(), GRB.MAXIMIZE)

In [12]:
# guardado del modelo para inspección
m.write('LAP.lp')
m.display()

Maximize
3459.4500000000007 assign[Lote 1,Mante] + 4673.700000000001 assign[Lote 1,Panuco]
+ 4259.75 assign[Lote 2,Mante] + 2893.5 assign[Lote 2,Panuco]
+ 7518.0 assign[Lote 3,Mante] + 12688.0 assign[Lote 3,Panuco]
+ 544.1400000000003 assign[Lote 4,Mante] + 1773.2399999999998 assign[Lote 4,Panuco] +
-724.1779999999999 assign[Lote 5,Mante] + 2774.2519999999995 assign[Lote 5,Panuco]
+ 4057.29 assign[Lote 6,Mante] + 571.1400000000003 assign[Lote 6,Panuco]
+ 5110.200000000001 assign[Lote 7,Mante] + 9173.2 assign[Lote 7,Panuco] +
-1602.4099999999999 assign[Lote 8,Mante] + 730.9399999999987 assign[Lote 8,Panuco]
+ 42.76000000000022 assign[Lote 9,Mante] + 1182.1599999999999 assign[Lote 9,Panuco]
+ 1944.1400000000003 assign[Lote 10,Mante] + 2613.24 assign[Lote 10,Panuco] +
-101.0 gap[Lote 1] + -101.0 gap[Lote 2] + -101.0 gap[Lote 3] + -101.0 gap[Lote 4] +
-101.0 gap[Lote 5] + -101.0 gap[Lote 6] + -101.0 gap[Lote 7] + -101.0 gap[Lote 8] +
-101.0 gap[Lote 9] + -101.0 gap[Lote 10]
Subject To
  lo

### Optimización

In [13]:
# correr el motor de optimización
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10 rows, 30 columns and 30 nonzeros
Model fingerprint: 0xa81ddb18
Variable types: 10 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 24609.142000
Presolve removed 10 rows and 30 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 43925.8 24609.1 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.392577200000e+04, best bound 4.392577200000e+04, gap 0.0000%


In [14]:
# despliegue de las variables de decisión que lograron un valor óptimo
for v in m.getVars():
	if (abs(v.x) > 1e-6):
		print(v.varName, v.x)

# desplegar el revenue total alcanzado
print('Valor óptimo de la función objetivo:', round(m.objVal,2))

# Calculo del revenue total basado en las variables de asignación
total_revenue = 0
for [r, j] in combinations:
    if (abs(x[r, j].x) > 1e-6):
        total_revenue = total_revenue + revenue[r, j]*x[r, j].x

print('Revenue total logrado: ', round(total_revenue,2))


assign[Lote 1,Panuco] 1.0
assign[Lote 2,Mante] 1.0
assign[Lote 3,Panuco] 1.0
assign[Lote 4,Panuco] 1.0
assign[Lote 5,Panuco] 1.0
assign[Lote 6,Mante] 1.0
assign[Lote 7,Panuco] 1.0
assign[Lote 8,Panuco] 1.0
assign[Lote 9,Panuco] 1.0
assign[Lote 10,Panuco] 1.0
Valor óptimo de la función objetivo: 43925.77
Revenue total logrado:  43925.77
